In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import time
from utils import *
from models import *
from attention import *

import torch
import numpy as np
import torch.optim as optim
from torch.utils import data
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Download the file
get_file()
path_to_file = 'spa-eng/spa.txt'

======== THE DATASET IS READY ========


In [4]:
# Try experimenting with the size of that dataset
num_examples = 118960
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

The number of line in the dataset is 118964


In [5]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

107064 107064 11896 11896


In [6]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
2 ----> <start>
7 ----> el
15 ----> es
10172 ----> futbolista
4 ----> .
3 ----> <end>

Target Language; index to word mapping
2 ----> <start>
13 ----> he
12 ----> is
10 ----> a
860 ----> soccer
1137 ----> player
4 ----> .
3 ----> <end>


In [7]:
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)// BATCH_SIZE
embedding_dim_source = 256
embedding_dim_target = 100
units = 1024
vocab_inp_size = len(inp_lang.word_index) + 1
vocab_tar_size = len(targ_lang.word_index) + 1

In [8]:
%%time

glove_dir = '../glove_weights/'

embeddings_index = {}
with open(os.path.join(glove_dir, 'glove.6B.100d.txt'), 'r', encoding='utf-8') as f:
    lines = f.readlines()
    
for line in lines:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))


embedding_dim = 100
embedding_matrix = np.zeros((vocab_tar_size, embedding_dim))
for word, i in targ_lang.word_index.items():
    if i < vocab_tar_size:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Found 400000 word vectors.
CPU times: user 13.6 s, sys: 267 ms, total: 13.9 s
Wall time: 13.9 s


In [9]:
# transform to torch tensor
tensor_x = torch.Tensor(input_tensor_train).long() 
tensor_y = torch.Tensor(target_tensor_train).long()
# create your datset
my_dataset = data.TensorDataset(tensor_x,tensor_y) 
# create your dataloader
my_dataloader = data.DataLoader(my_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        drop_last=True,
                        num_workers=4)   

In [10]:
example_input_batch, example_target_batch = next(iter(my_dataloader))

In [11]:
example_input_batch.shape

torch.Size([64, 42])

In [12]:
encoder = Encoder(vocab_inp_size, embedding_dim_source, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) torch.Size([64, 42, 1024])
Encoder Hidden state shape: (batch size, units) torch.Size([1, 64, 1024])


In [13]:
attention_layer = BahdanauAttention(10, 1024)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) torch.Size([64, 1024])
Attention weights shape: (batch_size, sequence_length, 1) torch.Size([64, 42, 1])


In [14]:
decoder = Decoder(vocab_tar_size, embedding_dim_target, units, BATCH_SIZE, 1024)

sample_decoder_output, _, _ = decoder(torch.randint(1, 20, (BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) torch.Size([64, 13853])


In [15]:
encoder = Encoder(vocab_inp_size, embedding_dim_source, units, BATCH_SIZE).to(device)
decoder = Decoder(vocab_tar_size, embedding_dim_target, units, BATCH_SIZE, 1024).to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [16]:
# Setting the pretrained glove's weights in the Decoder
decoder.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, device=device).float())

In [ ]:
# Train the model
EPOCHS = 20

for epoch in range(EPOCHS):
    start = time.time()

    total_loss = 0

    for (batch, (inp, targ)) in enumerate(my_dataloader):
        inp, targ = inp.to(device), targ.to(device)
        batch_loss = train_step(inp, targ, encoder, decoder,
                                encoder_optimizer, decoder_optimizer,
                                criterion, device, BATCH_SIZE, targ_lang)
        
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss))
            
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        
        pass

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


Epoch 1 Batch 0 Loss 9.3025
Epoch 1 Batch 100 Loss 1.7025
Epoch 1 Batch 200 Loss 1.5339
Epoch 1 Batch 300 Loss 1.2020
Epoch 1 Batch 400 Loss 1.0292
Epoch 1 Batch 500 Loss 1.0688
Epoch 1 Batch 600 Loss 1.2681
Epoch 1 Batch 700 Loss 0.9368
Epoch 1 Batch 800 Loss 1.2883
Epoch 1 Batch 900 Loss 1.0381
Epoch 1 Batch 1000 Loss 0.9625
Epoch 1 Batch 1100 Loss 0.9252
Epoch 1 Batch 1200 Loss 0.9168
Epoch 1 Batch 1300 Loss 0.8842
Epoch 1 Batch 1400 Loss 1.1590
Epoch 1 Batch 1500 Loss 1.3492
Epoch 1 Batch 1600 Loss 0.7430
Epoch 1 Loss 1.2110
Time taken for 1 epoch 1076.4737224578857 sec

Epoch 2 Batch 0 Loss 0.7252
Epoch 2 Batch 100 Loss 1.5175
Epoch 2 Batch 200 Loss 0.8572
Epoch 2 Batch 300 Loss 0.7244
Epoch 2 Batch 400 Loss 1.2360
Epoch 2 Batch 500 Loss 0.7041
Epoch 2 Batch 600 Loss 0.9075
Epoch 2 Batch 700 Loss 0.7779
Epoch 2 Batch 800 Loss 0.7562
Epoch 2 Batch 900 Loss 0.8102
Epoch 2 Batch 1000 Loss 0.5955
Epoch 2 Batch 1100 Loss 0.6611
Epoch 2 Batch 1200 Loss 0.9849
Epoch 2 Batch 1300 Loss 0.7

In [ ]:
translate(u'trata de averiguarlo .', max_length_targ, max_length_inp, encoder,
          decoder, inp_lang, targ_lang, device, beam_width=10, alpha=0.1)

In [ ]:
translate(u'trata de averiguarlo .', max_length_targ, max_length_inp, encoder,
          decoder, inp_lang, targ_lang, device, beam_width=100, alpha=0.1)

In [ ]:
translate(u'trata de averiguarlo .', max_length_targ, max_length_inp, encoder,
          decoder, inp_lang, targ_lang, device, beam_search=False)

In [ ]:
translate(u'esta es mi vida .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device)

In [ ]:
translate(u'esta es mi vida .', max_length_targ, max_length_inp, encoder,
          decoder, inp_lang, targ_lang, device, beam_width=10, alpha=0.2)

In [ ]:
translate(u'hace mucho frio aqui .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device, beam_search=False)

In [ ]:

translate(u'hace mucho frio aqui .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device, beam_width=5, alpha=0.2)

# FIN